<a href="https://colab.research.google.com/github/OliviaHutchison/IMLOProject/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports + Setup

In [1]:
#check for GPU access with PyTorch
import torch
torch.cuda.is_available()

True

In [2]:
#setup DAC
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
#Import PyTorch
import torch
from torch import nn

#Torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize

#Visualisation
import matplotlib.pyplot as plt

#Version check
print(torch.__version__)
print(torchvision.__version__)

2.2.1+cu121
0.17.1+cu121


# Get Dataset

In [4]:
transform = transforms.Compose([
    transforms.RandomAutocontrast(0.5),
    transforms.RandomAdjustSharpness(4, 0.5),
    transforms.RandomPosterize(4, 0.5),
    transforms.GaussianBlur(3),
    transforms.RandomRotation(90),
    ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),
    Resize((256, 256))
])

In [5]:
#Training Version
train_data= datasets.Flowers102(
    root="data", #where to download to
    split='train', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

In [6]:
#Validation
val_data= datasets.Flowers102(
    root="data", #where to download to
    split='val', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

In [7]:
#Testing
test_data= datasets.Flowers102(
    root="data", #where to download to
    split='test', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

In [8]:
'''
#View Some data
torch.manual_seed(42)
fig = plt.figure(figsize=(9,9))
rows, cols = 4, 4
for i in range(1, rows*cols+1):
  randIndex = torch.randint(0, len(train_data), size=[1]).item()
  #print(randIndex)
  image, label = train_data[randIndex]
  fig.add_subplot(rows, cols, i)
  #change dims from colour, height, width to height, width, col
  image = image.permute(1,2,0)
  plt.imshow(image, cmap="gray")
  plt.title(label)
  plt.axis(False);

'''

'\n#View Some data\ntorch.manual_seed(42)\nfig = plt.figure(figsize=(9,9))\nrows, cols = 4, 4\nfor i in range(1, rows*cols+1):\n  randIndex = torch.randint(0, len(train_data), size=[1]).item()\n  #print(randIndex)\n  image, label = train_data[randIndex]\n  fig.add_subplot(rows, cols, i)\n  #change dims from colour, height, width to height, width, col\n  image = image.permute(1,2,0)\n  plt.imshow(image, cmap="gray")\n  plt.title(label)\n  plt.axis(False);\n\n'

# Prep Data

In [9]:
from torch.utils.data import DataLoader

In [10]:
batchSize = 32 #groups of 32 images

#Turn data into iterables
trainDataloader = DataLoader(dataset=train_data,
                             batch_size=batchSize,
                             shuffle=True)

valDataloader = DataLoader(dataset=val_data,
                            batch_size=batchSize,
                            shuffle=False)

testDataloader = DataLoader(dataset=test_data,
                            batch_size=batchSize,
                            shuffle=False)

In [11]:
#trainFeaturesBatch, trainLabelsBatch = next(iter(trainDataloader))
#trainFeaturesBatch.shape


In [12]:
'''
#Show sample
torch.manual_seed(42)
randomIndex = torch.randint(0, len(trainFeaturesBatch), size=[1]).item()
img, label = trainFeaturesBatch[randomIndex], trainLabelsBatch[randomIndex]
img = img.permute(1,2,0)
plt.imshow(img, cmap="gray")
plt.title(label)
plt.axis(False)
'''

'\n#Show sample\ntorch.manual_seed(42)\nrandomIndex = torch.randint(0, len(trainFeaturesBatch), size=[1]).item()\nimg, label = trainFeaturesBatch[randomIndex], trainLabelsBatch[randomIndex]\nimg = img.permute(1,2,0)\nplt.imshow(img, cmap="gray")\nplt.title(label)\nplt.axis(False)\n'

# CNN

In [13]:
#Create CNN
class FlowersModel(nn.Module):
  def __init__(self, inputShape: int, hiddenUnits: int, outputShape: int):
    super().__init__()
    #convolutional block 1 - feature extractor
    self.conv_block1 = nn.Sequential(
        #create conv layer
        nn.Conv2d(in_channels=inputShape, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1), #condv2d for 2d data
        #values we set ourselves in NNs are hyperparameters

        #ReLu Layer
        nn.ReLU(),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #convolutional block 2 - feature extractor
    self.conv_block2 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #convolutional block 3 - feature extractor
    self.conv_block3 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #Dropout Layer
        nn.Dropout(0.3),
    )
    #convolutional block 4 - feature extractor
    self.conv_block4 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #convolutional block 5 - feature extractor
    self.conv_block5 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #classifier layer
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hiddenUnits*64*64,
                  out_features=outputShape),
        nn.Softmax(dim=1),
        nn.Linear(in_features=hiddenUnits,
                  out_features=outputShape)
    )

  #forward
  def forward(self, x):
    x = self.conv_block1(x)
    #print(x.shape)
    x = self.conv_block2(x)
    #print(x.shape)
    x = self.classifier(x)
    return x


In [14]:
model1 = FlowersModel(inputShape=3,
                      hiddenUnits=102,
                      outputShape=102).to(device)

## Loss and Optimisation

In [15]:
##Loss and Optimiser
lossCalc = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(params=model1.parameters(),
                             lr=0.0001)

## Save Model

# Test and Training

## Setup and functions

In [16]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

### Timer

In [17]:
##Create Timer
from timeit import default_timer as timer
def printTime(start: float, end: float, device: torch.device = None):
    totalTime = end - start
    minutes = totalTime // 60
    print(f"Train time on {device}: {totalTime:.3f} seconds.\n Roughly {minutes} minutes")
    return totalTime

### Accuracy

In [18]:
#Accuaracy Calculator
def accuracyCalc(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

### Training

In [19]:
#Training Function
def trainFunction(model: torch.nn.Module,
               dataLoader: torch.utils.data.DataLoader,
               lossCalc: torch.nn.Module,
               optimiser: torch.optim.Optimizer,
               accuracyCalc,
               device: torch.device = device):
    trainLoss, trainAcc = 0, 0
    for batch, (X, y) in enumerate(dataLoader):
        #Send data to GPU
        X, y = X.to(device), y.to(device)

        #Forward pass
        y_pred = model(X)

        #Calculate loss
        loss = lossCalc(y_pred, y)
        trainLoss += loss
        trainAcc += accuracyCalc(y_true=y,
                                 y_pred=y_pred.argmax(dim=1))

        #Optimizer zero grad
        optimiser.zero_grad()

        #Backpropagation
        loss.backward()

        #Optimiser step
        optimiser.step()

    # Calculate loss and accuracy per epoch
    trainLoss /= len(dataLoader)
    trainAcc /= len(dataLoader)
    print(f"Train loss: {trainLoss:.5f} | Train accuracy: {trainAcc:.2f}%")

### Validation

In [20]:
#Validation Function
def valFunction(dataLoader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              lossCalc: torch.nn.Module,
              accuracyCalc,
              device: torch.device = device):
    valLoss, valAcc = 0, 0
    model.eval()
    #Turn on inference context manager
    with torch.inference_mode():
        for X, y in dataLoader:
            #Send data to GPU
            X, y = X.to(device), y.to(device)

            #Forward pass
            val_pred = model(X)

            #Calculate loss and accuracy
            valLoss += lossCalc(val_pred, y)
            valAcc += accuracyCalc(y_true=y,
                y_pred=val_pred.argmax(dim=1)
            )

        #Adjust and display
        valLoss /= len(dataLoader)
        valAcc /= len(dataLoader)
        print(f"Val loss: {valLoss:.5f} | Val accuracy: {valAcc:.2f}%\n")

### Test

In [21]:
#Testing Function
def testFunction(dataLoader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              lossCalc: torch.nn.Module,
              accuracyCalc,
              device: torch.device = device):
    testLoss, testAcc = 0, 0
    model.eval()
    #Turn on inference context manager
    with torch.inference_mode():
        for X, y in dataLoader:
            #Send data to GPU
            X, y = X.to(device), y.to(device)

            #Forward pass
            test_pred = model(X)

            #Calculate loss and accuracy
            testLoss += lossCalc(test_pred, y)
            testAcc += accuracyCalc(y_true=y,
                y_pred=test_pred.argmax(dim=1)
            )

        #Adjust and display
        testLoss /= len(dataLoader)
        testAcc /= len(dataLoader)
        print(f"Test loss: {testLoss:.5f} | Test accuracy: {testAcc:.2f}%\n")

## Train and Test

In [ ]:
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
trainTimeStartModel1 = timer()

# Train and test model
epochs = 20
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    trainFunction(dataLoader=trainDataloader,
        model=model1,
        lossCalc=lossCalc,
        optimiser=optimiser,
        accuracyCalc=accuracyCalc,
        device=device
    )
    valFunction(dataLoader=valDataloader,
        model=model1,
        lossCalc=lossCalc,
        accuracyCalc=accuracyCalc,
        device=device
    )


testFunction(dataLoader=testDataloader,
    model=model1,
    lossCalc=lossCalc,
    accuracyCalc=accuracyCalc,
    device=device
)


trainTimeEndModel1 = timer()
totalTrainTime = printTime(start=trainTimeStartModel1,
                           end=trainTimeEndModel1,
                           device=device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 4.62764 | Train accuracy: 0.80%


## Save Model

1. `torch.save()` - saving and loading object, save in pythons's pickle format

2. `torch.load()` - load with pickle

3. `torch.nn.Module.load_state_dict()` - load models saved state dictionary

In [ ]:
torch.save(model1.state_dict(), './CNNModel.pth')

### Predictions


In [ ]:
def makePredictions(model: torch.nn.Module,
                    data: list,
                    device: torch.device = device):
  predProbs = []
  model.eval()
  with torch.inference_mode():
    for sample in data:
      #prep sample
      sample = torch.unsqueeze(sample, dim=0).to(device)

      #forward pass
      predLogit = model(sample)

      #prediction prob
      predProb = torch.softmax(predLogit.squeeze(), dim=0)

      #Get pred prob off GPU
      predProbs.append(predProb.cpu())
  #Stack all
  return torch.stack(predProbs)


In [ ]:
import random
random.seed(42)
testSamples = []
testLabels= []
for sample, label in random.sample(list(test_data), k=9):
  testSamples.append(sample)
  testLabels.append(label)



In [ ]:
#Make predictions
predProbs = makePredictions(model=model1,
                            data=testSamples)

predProbs[:10]

In [ ]:
#Convert to labels
predClass = predProbs.argmax(dim=1)

In [ ]:
plt.figure(figsize=(9,9))
nrows= 3
ncols = 3
for i, sample in enumerate(testSamples):
  plt.subplot(nrows, ncols, i+1)

  changedSample = sample.permute(1,2,0)
  plt.imshow(changedSample.squeeze())

  #Pred label
  predLabel = predClass[i]
  #Get truth label
  truthLabel = testLabels[i]

  #check for equality
  if truthLabel == predLabel:
    plt.title("True")
  else:
    plt.title(f"False. True Label: {truthLabel}, Pred:{predLabel}")

# End

Chris says mess with:
* Hyperparameters
* Transforms (as many as POSSIBLE :D)
* Optimisers (learning rate, weight decay, etc.)